In [4]:
from google.transit import gtfs_realtime_pb2
import requests
import pandas as pd

In [5]:
feed = gtfs_realtime_pb2.FeedMessage()
response = requests.get('https://bustime.ttc.ca/gtfsrt/vehicles')
feed.ParseFromString(response.content)
current_time = pd.Timestamp.now().time()

In [6]:
def parse_feed_and_filter_by_route(target_route_id):
  print(target_route_id)
  matching_entities = []
  for entity in feed.entity:
      if entity.HasField("vehicle"):
        vehicle = entity.vehicle
        if vehicle.trip.route_id == str(target_route_id):
            print(vehicle.trip)
            matching_entities.append({
                "id": entity.id,
                "trip_id": vehicle.trip.trip_id,
                "route_id": vehicle.trip.route_id,
                "latitude": vehicle.position.latitude,
                "longitude": vehicle.position.longitude,
                "occupancy_status": vehicle.occupancy_status
            })
  return matching_entities

In [7]:
target_route_id = 510
output = 'data_out'
vehicles_on_route = parse_feed_and_filter_by_route(target_route_id)

510
trip_id: "5656040"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "140915040"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "183869040"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "142137040"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "152617040"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "113948040"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "185987040"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "82453040"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "188414040"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "30512040"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "133742040"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "49523040"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "157917040"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "9392040"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: 

In [ ]:
# Mapping from dictionary keys to desired column names
key_to_column = {
    "id": "vehicle_id",
    "trip_id": "trip_id",
    "route_id": "route_id",
    "latitude": "vehicle_latitude",
    "longitude": "vehicle_longitude"
}

# Create the DataFrame, remove duplicate vehicles
df = pd.DataFrame(
    [{new_key: d.get(old_key) for old_key, new_key in key_to_column.items()} for d in vehicles_on_route])
df.drop_duplicates()

# Add current timestampt to df
df['now'] = current_time

# Count unique trip_ids
unique_trip_count = df["trip_id"].nunique()
# print(unique_trip_count)

# Save to csv
df.to_csv(output + '/' + str(target_route_id) + "-rt.csv")